In [ ]:
#Collect 1000 images for face recognition
import cv2
import numpy as np

# Load haarcascade face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray  = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        crop_face = img[y:y+h, x:x+w]

    return crop_face

# Start Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 1000 samples of your face from webcam input
while True:


    status, img = cap.read()
    if face_extractor(img) is not None:
        count += 1
        face = cv2.resize(face_extractor(img), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save face in FaceRecognition directory with unique name
        file_name_path = 'C://Users//admin//Desktop//FaceRecognition//MyImage//Test_set//Dharmesh//' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)


        # Show live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('MyImage', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 34:
        break
        #13 is for enter key
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

In [1]:
from keras.applications import VGG16


# VGG16 works on 224 x 224 pixels.
img_rows = 224
img_cols = 224 


# load VGG16 pre-trained model without the top layer.
# we are using predefined weights of imagenet
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we are freezing all the layers
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

Using TensorFlow backend.


0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [2]:
def addModel(model, num_classes, D=128):
#add top layers in old model
    top_model = model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [3]:
model.layers

In [4]:
from keras.layers import Dense, Dropout,Conv2D, MaxPooling2D, ZeroPadding2D, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

new  = addModel(model, num_classes)

new_model = Model(inputs=model.input, outputs=new )

print(new_model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [5]:
#using image augmentation
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'C:/Users/admin/Desktop/FaceRecognition/MyImage/Train_set'
validation_data_dir = 'C:/Users/admin/Desktop/FaceRecognition/MyImage/Test_set'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 2000 images belonging to 2 classes.
Found 68 images belonging to 2 classes.


In [6]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("facedetect.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
new_model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1000
nb_validation_samples = 150
epochs = 3
batch_size = 16

history = new_model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save("facedetect.h5.h5")

Epoch 1/3
62/62 [==============================] - 677s 11s/step - loss: 1.0263 - accuracy: 0.8619 - val_loss: 0.0931 - val_accuracy: 0.9432

Epoch 00001: val_loss improved from inf to 0.09310, saving model to facedetect.h5
Epoch 2/3
62/62 [==============================] - 663s 11s/step - loss: 0.0731 - accuracy: 0.9778 - val_loss: 0.0056 - val_accuracy: 1.0000

Epoch 00002: val_loss improved from 0.09310 to 0.00562, saving model to facedetect.h5
Epoch 3/3
62/62 [==============================] - 673s 11s/step - loss: 0.0635 - accuracy: 0.9859 - val_loss: 0.0023 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.00562 to 0.00229, saving model to facedetect.h5


In [7]:
train_generator.class_indices

{'Akshit': 0, 'Dharmesh': 1}

In [8]:
validation_generator.class_indices

{'Akshit': 0, 'Dharmesh': 1}